In [1]:
#Imports
import pandas as pd
import requests
import json

In [2]:
DarkPattern_dataset=pd.read_csv('/kaggle/input/dpbh-8-dp-dataset/final.csv')

In [3]:
DarkPattern_dataset.head()

,text,label
0,"Hurry, only 3 items left in stock!",false_urgency
1,Limited time offer! Get 20% off if you buy in ...,false_urgency
2,Don't miss out! This offer expires soon.,false_urgency
3,Last chance to save! This deal ends at midnight.,false_urgency
4,Order now before it's too late!,false_urgency


In [4]:
from sklearn.preprocessing import LabelEncoder

# Assuming 'data' is your DataFrame
label_encoder = LabelEncoder()
DarkPattern_dataset['target'] = label_encoder.fit_transform(DarkPattern_dataset['label'])

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [5]:
label_encoder.inverse_transform([0,1,2,3,4,5,6,7])

array(['confirm_shaming', 'disguised_ads', 'false_urgency',
       'forced_action', 'not dark pattern', 'rogue_malware',
       'sneaky_subscriptions', 'trick_questions'], dtype=object)

In [6]:
DarkPattern_dataset['target'].value_counts()

target
7    3650
6    3544
5    3312
3    3186
4    2778
0    2715
2    2372
1    2295
Name: count, dtype: int64

#Fine-Tuning Bert Model (Binary Classification)

In [7]:
DarkPattern_dataset.head()

,text,label,target
0,"Hurry, only 3 items left in stock!",false_urgency,2
1,Limited time offer! Get 20% off if you buy in ...,false_urgency,2
2,Don't miss out! This offer expires soon.,false_urgency,2
3,Last chance to save! This deal ends at midnight.,false_urgency,2
4,Order now before it's too late!,false_urgency,2


In [8]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
import torch
from transformers import TrainingArguments, Trainer
from transformers import BertTokenizer, BertForSequenceClassification

In [9]:
from transformers import BertTokenizer, BertForSequenceClassification
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=8)

tokenizer_config.json:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [10]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [11]:
model = model.to('cuda')

In [12]:
#Testing Tokenizer
sample_data = ["I am eating","I am playing "]
tokenizer(sample_data, padding=True, truncation=True, max_length=512)

{'input_ids': [[101, 1045, 2572, 5983, 102], [101, 1045, 2572, 2652, 102]], 'token_type_ids': [[0, 0, 0, 0, 0], [0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1], [1, 1, 1, 1, 1]]}

In [13]:
X = list(DarkPattern_dataset["text"])
y = list(DarkPattern_dataset["target"])
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2,stratify=y)
X_train_tokenized = tokenizer(X_train, padding=True, truncation=True, max_length=512)
X_val_tokenized = tokenizer(X_val, padding=True, truncation=True, max_length=512)

In [14]:
X_train_tokenized.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [15]:
print(X_train_tokenized['attention_mask'][0])

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 

In [16]:
len(X_train),len(X_val)

(19081, 4771)

In [17]:
# Create torch dataset
class Dataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels=None):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        if self.labels:
            item["labels"] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.encodings["input_ids"])

In [18]:
train_dataset = Dataset(X_train_tokenized, y_train)
val_dataset = Dataset(X_val_tokenized, y_val)

In [19]:
train_dataset[5]

{'input_ids': tensor([ 101, 4081, 3463,  102,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0, 

In [20]:
def compute_metrics(p):
    print(type(p))
    pred, labels = p
    pred = np.argmax(pred, axis=1)

    accuracy = accuracy_score(y_true=labels, y_pred=pred)
    recall = recall_score(y_true=labels, y_pred=pred,average='macro')
    precision = precision_score(y_true=labels, y_pred=pred,average='macro')
    f1 = f1_score(y_true=labels, y_pred=pred,average='macro')

    return {"accuracy": accuracy, "precision": precision,"recall":recall, "f1": f1}

In [21]:
# Define Trainer
args = TrainingArguments(
    output_dir="/kaggle/working/",
    num_train_epochs=30,
    per_device_train_batch_size=8,
    report_to='none',
    save_strategy='no'

)
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [22]:
trainer.train()

Step,Training Loss
500,0.856100
1000,0.618100
1500,0.586600
2000,0.518200
2500,0.523000
3000,0.477900
3500,0.447100
4000,0.440900
4500,0.470200
5000,0.450500


TrainOutput(global_step=71580, training_loss=0.20245686127973622, metrics={'train_runtime': 31189.4812, 'train_samples_per_second': 18.353, 'train_steps_per_second': 2.295, 'total_flos': 1.5062077516161024e+17, 'train_loss': 0.20245686127973622, 'epoch': 30.0})

In [23]:
trainer.evaluate()

<class 'transformers.trainer_utils.EvalPrediction'>


{'eval_loss': 1.2968487739562988,
 'eval_accuracy': 0.8398658562146301,
 'eval_precision': 0.8498045987887886,
 'eval_recall': 0.8463839020589946,
 'eval_f1': 0.8479566188481811,
 'eval_runtime': 83.9185,
 'eval_samples_per_second': 56.853,
 'eval_steps_per_second': 7.114,
 'epoch': 30.0}

In [24]:
trainer.save_model('Bert-8dp-30e')

In [25]:
text = "Hurry up and buy discount ends today"
inputs = tokenizer(text,padding = True, truncation = True, return_tensors='pt').to('cuda')
outputs = model(**inputs)
print(outputs)
predictions = torch.nn.functional.softmax(outputs.logits, dim=-1)
print(predictions)
predictions = predictions.cpu().detach().numpy()
predictions=np.argmax(predictions,axis=1)

SequenceClassifierOutput(loss=None, logits=tensor([[-1.9347, -1.5940,  8.9858,  0.0510, -2.0259, -2.0107, -2.0764, -0.6779]],
       device='cuda:0', grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)
tensor([[1.8078e-05, 2.5418e-05, 9.9971e-01, 1.3169e-04, 1.6503e-05, 1.6756e-05,
         1.5690e-05, 6.3535e-05]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [26]:
classes=['confirm_shaming', 'disguised_ads', 'false_urgency',
       'forced_action', 'not dark pattern', 'rogue_malware',
       'sneaky_subscriptions', 'trick_questions']

In [27]:
pred_class=classes[predictions[0]]

In [28]:
print(pred_class)

false_urgency
